# Представление изображений и предобработка

In [ ]:
!pip install opencv-python -q

In [7]:
import cv2
import requests
import numpy as np
import matplotlib.pyplot as plt

## Представление изображений

Цифровое изображение можно рассматривать как двумерную матрицу, где каждый элемент соответствует пикселю. В случае цветных изображений мы имеем три матрицы (обычно для каналов R, G, B). Если $ I(x,y) $ обозначает яркость пикселя в точке $(x,y)$, то общее изображение представляется как:

$$
I = \{ I(x,y) \,|\, x = 1,\ldots, M; \; y = 1,\ldots, N \}
$$

где $ M $ и $ N $ – размеры изображения.

### Гистограмма изображения

Гистограмма изображений – это распределение яркости пикселей. Для каждого уровня интенсивности $ i $ (например, от 0 до 255 для 8-битных изображений) можно определить гистограмму $ H(i) $ следующим образом:

$$
H(i) = \sum_{x=1}^{M} \sum_{y=1}^{N} \delta(I(x,y) - i)
$$

где $ \delta $ – дельта-функция Кронекера, равная 1, если $ I(x,y) = i $, и 0 в противном случае.

Гистограмма помогает понять распределение яркости, выявить проблемы контраста или насыщенности, а также служит основой для таких алгоритмов, как выравнивание гистограммы.

### Фильтры и свёртка

Фильтры применяются для улучшения качества изображения, выделения структур или подавления шума. Одним из основных методов является свёртка изображения с ядром фильтра. Если $ h(u,v) $ – ядро свёртки, то результирующее изображение $ I'(x,y) $ вычисляется по формуле:

$$
I'(x,y) = \sum_{u=-k}^{k} \sum_{v=-k}^{k} I(x+u, y+v) \cdot h(u,v)
$$

где $ k $ определяется размером фильтра (например, для фильтра $3 \times 3$ $ k=1 $).  
**Примеры фильтров и результатов их применения:**

| **Формула свёртки** | **Результат применения** |
|---------------------|--------------------------|
| **Identity Filter** <br> $ \begin{bmatrix} 0 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 0 \end{bmatrix} $ | Оставляет изображение без изменений – используется для проверки корректности алгоритма свёртки. |
| **Mean Filter (Box Blur)** <br> $ \frac{1}{9}\begin{bmatrix} 1 & 1 & 1 \\ 1 & 1 & 1 \\ 1 & 1 & 1 \end{bmatrix} $ | Сглаживает изображение, уменьшая шум, но также смягчает мелкие детали и края. |
| **Gaussian Filter** <br> $ \frac{1}{16}\begin{bmatrix} 1 & 2 & 1 \\ 2 & 4 & 2 \\ 1 & 2 & 1 \end{bmatrix} $ | Обеспечивает сглаживание с меньшей потерей резкости, так как веса распределяются по нормальному закону – лучше сохраняет границы объектов по сравнению с средним фильтром. |
| **Sobel Operator (Horizontal)** <br> $ \begin{bmatrix} -1 & 0 & 1 \\ -2 & 0 & 2 \\ -1 & 0 & 1 \end{bmatrix} $ | Выделяет вертикальные края, обнаруживая резкие изменения интенсивности по горизонтали. |
| **Sobel Operator (Vertical)** <br> $ \begin{bmatrix} -1 & -2 & -1 \\ 0 & 0 & 0 \\ 1 & 2 & 1 \end{bmatrix} $ | Выделяет горизонтальные края, фиксируя вертикальные изменения яркости. |
| **Laplacian Filter** <br> $ \begin{bmatrix} 0 & 1 & 0 \\ 1 & -4 & 1 \\ 0 & 1 & 0 \end{bmatrix} $ | Подчеркивает области резкого изменения яркости, эффективно обнаруживая края, но может быть чувствителен к шуму. |
| **Sharpening Filter** <br> $ \begin{bmatrix} 0 & -1 & 0 \\ -1 & 5 & -1 \\ 0 & -1 & 0 \end{bmatrix} $ | Усиливает контраст и чёткость изображения, выделяя детали за счёт усиления резких переходов. |

In [5]:
identity_kernel = np.array([[0, 0, 0],
                            [0, 1, 0],
                            [0, 0, 0]], dtype=np.float32)

box_kernel = np.ones((3, 3), dtype=np.float32) / 9.0

gaussian_kernel = np.array([[1, 2, 1],
                            [2, 4, 2],
                            [1, 2, 1]], dtype=np.float32) / 16.0

sobel_horizontal = np.array([[-1, 0, 1],
                             [-2, 0, 2],
                             [-1, 0, 1]], dtype=np.float32)

sobel_vertical = np.array([[-1, -2, -1],
                           [ 0,  0,  0],
                           [ 1,  2,  1]], dtype=np.float32)

laplacian_kernel = np.array([[0, 1, 0],
                             [1, -4, 1],
                             [0, 1, 0]], dtype=np.float32)

sharpening_kernel = np.array([[0, -1, 0],
                              [-1, 5, -1],
                              [0, -1, 0]], dtype=np.float32)

In [ ]:
url = "https://raw.githubusercontent.com/opencv/opencv/master/samples/data/lena.jpg"
response = requests.get(url)

image_data = np.asarray(bytearray(response.content), dtype="uint8")
image = cv2.imdecode(image_data, cv2.IMREAD_GRAYSCALE)

identity_image    = cv2.filter2D(image, -1, identity_kernel)
box_blur_image    = cv2.filter2D(image, -1, box_kernel)
gaussian_image    = cv2.filter2D(image, -1, gaussian_kernel)
sobel_h_image     = cv2.filter2D(image, -1, sobel_horizontal)
sobel_v_image     = cv2.filter2D(image, -1, sobel_vertical)
laplacian_image   = cv2.filter2D(image, -1, laplacian_kernel)
sharpened_image   = cv2.filter2D(image, -1, sharpening_kernel)

fig, axes = plt.subplots(2, 4, figsize=(20, 15))
axes = axes.flatten()

titles = ["Original", "Identity Filter", "Box Blur", "Gaussian Filter",
          "Sobel Horizontal", "Sobel Vertical", 
          "Laplacian Filter","Sharpening Filter"]

images = [image, identity_image, box_blur_image, gaussian_image,
          sobel_h_image, sobel_v_image, laplacian_image, sharpened_image]

for ax, img, title in zip(axes, images, titles):
    ax.imshow(img, cmap='gray')
    ax.set_title(title, fontsize=20)
    ax.axis('off')

for ax in axes[len(images):]:
    ax.axis('off')

plt.tight_layout()
plt.show()

### Добавление шума

Шум часто моделируется для тестирования алгоритмов обработки изображений, позволяет создать более реалистичные условия.

- **Gaussian Noise:**
    Если $ n(x,y) $ – шум, распределённый по нормальному закону с математическим ожиданием 0 и дисперсией $ \sigma^2 $, то зашумлённое изображение можно описать как:
    $$
    I_{noise}(x,y) = I(x,y) + n(x,y), \quad n(x,y) \sim \mathcal{N}(0, \sigma^2)
    $$  
    *Введение случайных колебаний яркости по нормальному распределению, имитируя электронные помехи.*

- **Salt-and-Pepper Noise:**  
    *Случайное замещение пикселей изображением белыми и чёрными точками, что моделирует выбросы ошибок при передаче данных.*

- **Speckle Noise:**  
    $$
    I_{noise}(x,y) = I(x,y) + I(x,y) \cdot n(x,y), \quad n(x,y) \sim \mathcal{N}(0, \sigma^2)
    $$  
    *Мультипликативный шум, который часто наблюдается в изображениях, полученных с использованием радаров или ультразвука.*

In [18]:
def gaussian_noise(image, 
                   mean=0, 
                   sigma=25):
    gauss = np.random.normal(mean, sigma, image.shape).astype('float32')
    noisy = image.astype('float32') + gauss

    noisy = np.clip(noisy, 0, 255).astype('uint8')
    
    return noisy

def salt_and_pepper_noise(image, 
                          salt_prob=0.01, 
                          pepper_prob=0.01):
    noisy = image.copy()
    
    num_salt = np.ceil(salt_prob * image.size)
    coords = [np.random.randint(0, i, int(num_salt)) for i in image.shape]
    noisy[coords[0], coords[1]] = 255
    
    num_pepper = np.ceil(pepper_prob * image.size)
    coords = [np.random.randint(0, i, int(num_pepper)) for i in image.shape]

    noisy[coords[0], coords[1]] = 0
    
    return noisy

def speckle_noise(image, 
                  sigma=0.25):
    
    noise = np.random.randn(*image.shape) * sigma
    noisy = image + image * noise
    noisy = np.clip(noisy, 0, 255).astype('uint8')

    return noisy

def calc_hist(img):
    return cv2.calcHist([img], [0], None, [256], [0, 256])

In [ ]:
gaussian_img = gaussian_noise(image)
saltpepper_img = salt_and_pepper_noise(image)
speckle_img = speckle_noise(image)

hist_original = calc_hist(image)
hist_gaussian = calc_hist(gaussian_img)
hist_saltpepper = calc_hist(saltpepper_img)
hist_speckle = calc_hist(speckle_img)

images = [image, gaussian_img, saltpepper_img, speckle_img]
hists  = [hist_original, hist_gaussian, hist_saltpepper, hist_speckle]
titles = ["Original", "Gaussian Noise", "Salt & Pepper Noise", "Speckle Noise"]

fig, axs = plt.subplots(4, 2, figsize=(10, 16))

for i in range(4):
    axs[i, 0].imshow(images[i], cmap='gray')
    axs[i, 0].set_title(titles[i])
    axs[i, 0].axis('off')
    
    axs[i, 1].plot(hists[i])
    axs[i, 1].set_title(f"{titles[i]} Histogram")
    axs[i, 1].set_xlabel("Intensity")
    axs[i, 1].set_ylabel("Frequency")

plt.tight_layout()
plt.show()